In [7]:
!pip install langchain-core
!pip install langchain-community
!pip install chromadb
!pip install --upgrade chromadb langchain_community httpx
!pip install PyPDF2
!pip install ollama

  Using cached httpx-0.27.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached pydantic-2.9.1-py3-none-any.whl.metadata (146 kB)
  Using cached pydantic_core-2.23.3-cp312-none-win_amd64.whl.metadata (6.7 kB)
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ---------------------------------------- 2.3/2.3 MB 33.6 MB/s eta 0:00:00
Using cached httpx-0.27.2-py3-none-any.whl (76 kB)
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   -------------------- ------------------- 0.5/1.0 MB 5.6 MB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 6.8 MB/s eta 0:00:00
Using cached pydantic-2.9.1-py3-none-any.whl (434 kB)
Using cached pydantic_core-2.23.3-cp312-none-win_amd64.whl (1.9 MB)
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.18.2
    Uninstalling pydantic_core-2.18.2:
      Successfully uninstalled pydantic_core-2.18.2
  Attempting uninstall: pydantic
    Found existing installation: pyda

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-groq 0.1.4 requires langchain-core<0.3,>=0.1.45, but you have langchain-core 0.3.0 which is incompatible.
langchain-huggingface 0.0.3 requires langchain-core<0.3,>=0.1.52, but you have langchain-core 0.3.0 which is incompatible.


In [16]:
!pip install pydantic==1.10.12

  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.9.1
    Uninstalling pydantic-2.9.1:
      Successfully uninstalled pydantic-2.9.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 4.43.0 requires pydantic>=2.0, but you have pydantic 1.10.12 which is incompatible.
langchain 0.3.0 requires pydantic<3.0.0,>=2.7.4, but you have pydantic 1.10.12 which is incompatible.
langchain-core 0.3.0 requires pydantic<3.0.0,>=2.5.2; python_full_version < "3.12.4", but you have pydantic 1.10.12 which is incompatible.
langchain-groq 0.1.4 requires langchain-core<0.3,>=0.1.45, but you have langchain-core 0.3.0 which is incompatible.
langchain-huggingface 0.0.3 requires langchain-core<0.3,>=0.1.52, but you have langchain-core 0.3.0 which is incompatible.
pydantic-settings 2.5.2 requires pydantic>=2.7.0, but you have pydantic 1.10.12 which is incompatible.


In [17]:
!pip install --upgrade langchain groq pydantic

  Using cached pydantic-2.9.1-py3-none-any.whl.metadata (146 kB)
Using cached pydantic-2.9.1-py3-none-any.whl (434 kB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.12
    Uninstalling pydantic-1.10.12:
      Successfully uninstalled pydantic-1.10.12


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-groq 0.1.4 requires langchain-core<0.3,>=0.1.45, but you have langchain-core 0.3.0 which is incompatible.
langchain-huggingface 0.0.3 requires langchain-core<0.3,>=0.1.52, but you have langchain-core 0.3.0 which is incompatible.


In [19]:
!pip install groq

In [11]:
!pip install -qU langchain-groq

In [25]:
!pip install -U langchain-ollama

In [12]:
!pip install ollama

In [13]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import Chroma
from chromadb.config import Settings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_community import embeddings
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
import ollama  # Ensure you have the correct library for Ollama



In [29]:
# Configure Chroma to use FastAPI
import chromadb  # Ensure this is imported
import requests

loader = PyPDFDirectoryLoader("data")
the_text = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(the_text)

# Update the settings to use the correct API implementation
settings = chromadb.config.Settings(
    chroma_api_impl="chromadb.api.fastapi.FastAPI",  # Use FastAPI implementation
    chroma_server_host="localhost",
    chroma_server_http_port="8000"
)

# Initialize ChromaDB client with the correct settings
chroma_client = chromadb.Client(settings=settings)

# Use the correct model name
embedding = OllamaEmbeddings(model="llama3")

# Initialize the vectorstore
vectorstore = Chroma.from_documents(
    documents=docs,
    collection_name="ollama_embeds",
    embedding=embedding,
    client=chroma_client
)

retriever = vectorstore.as_retriever()


In [31]:
import os

# Set the environment variable
os.environ['GROQ_API_KEY'] = "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"


In [35]:
from groq import Groq
from langchain_groq import ChatGroq
from pydantic import BaseModel

llm = ChatGroq(
    model="llama3-8b-8192",
    temperature=0.0,
    max_retries=2,
    # other params...
)

In [36]:
rag_template = """Answer the question based only on the following context:
{context}
Question: {question}
"""
rag_prompt = ChatPromptTemplate.from_template(rag_template)
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

In [37]:
import time
import textwrap
import gradio as gr

# Test the architecture with a simple hard coded question
response = rag_chain.invoke("What is this document about")
print(textwrap.fill(response, width=80))

Based on the provided context, it appears that this document is about the
Chandrayaan 3 mission, specifically the operations and events that took place
during the mission.


In [38]:
# Make the questions dynamic using a chat interface. Let's use gradio for this.
def process_question(user_question):
    start_time = time.time()

    # Directly using the user's question as input for rag_chain.invoke
    response = rag_chain.invoke(user_question)

    # Measure the response time
    end_time = time.time()
    response_time = f"Response time: {end_time - start_time:.2f} seconds."

    # Combine the response and the response time into a single string
    full_response = f"{response}\n\n{response_time}"

    return full_response

# Setup the Gradio interface
iface = gr.Interface(fn=process_question,
                     inputs=gr.Textbox(lines=2, placeholder="Type your question here..."),
                     outputs=gr.Textbox(),
                     title="Personal Knowledge Chat App",
                     description="Ask any question about your document, and get an answer along with the response time.")

# Launch the interface
iface.launch(share=True)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://1b7b70c1b016d8d423.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
